In [1]:
import json
import xml.etree.ElementTree as ElementTree

import numpy as np
import pandas as pd
import requests
from pyproj import Proj

In [2]:
# read key from key.json file 
PUBLIC_DATA_API_KEY = json.loads(open('key.json').read())['LX']
SRS_NAME = 'EPSG:5179'

In [3]:
proj = Proj(SRS_NAME)

In [4]:
# left_bottom_lonlat = (126.97999191237074, 37.55913710608509)
# right_top_lonlat = (126.9851453896948, 37.56312770042335)
left_bottom_lonlat = (126.979, 37.559)
right_top_lonlat = (126.980, 37.560)

In [5]:
GIS_BUILDING_SERVICE_WFS = 'https://apis.data.go.kr/1611000/nsdi/GisBuildingService/wfs/getGisGnrlBuildingWFS'
TYPE_NAME = "F171"
MAX_FEATURES = "100"
RESULT_TYPE = "results"
bbox_epsg = ','.join(map(str, [*proj(*left_bottom_lonlat), *proj(*right_top_lonlat)]))
params = {
    'serviceKey' : PUBLIC_DATA_API_KEY,
    'typename' : TYPE_NAME,
    'bbox' : bbox_epsg + ',' + SRS_NAME,
    'maxFeatures' : MAX_FEATURES,
    'resultType' : RESULT_TYPE,
    'srsName' : SRS_NAME
}

response = requests.get(GIS_BUILDING_SERVICE_WFS, params=params)

In [6]:
response_text = response.content.decode()
print(response_text)

<?xml version="1.0" encoding="utf-8" ?>
<wfs:FeatureCollection xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:wfs="http://www.opengis.net/wfs" xmlns:gml="http://www.opengis.net/gml" xmlns:NSDI="http://10.1.17.66:6080/arcgis/services/opend/EiosSpceServiceWFS/MapServer/WFSServer" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" timeStamp="2023-09-19T09:45:05Z" numberOfFeatures="unknown" xsi:schemaLocation="http://www.opengis.net/wfs http://schemas.opengis.net/wfs/1.1.0/wfs.xsd http://www.opengis.net/gml http://schemas.opengis.net/gml/3.1.1/base/gml.xsd http://10.1.17.66:6080/arcgis/services/opend/EiosSpceServiceWFS/MapServer/WFSServer http://10.1.17.66:6080/arcgis/services/opendb/EiosSpceServiceWFS/MapServer/WFSServer?service=wfs%26version=1.1.0%26request=DescribeFeatureType">
<gml:boundedBy>
  <gml:Envelope srsName="urn:ogc:def:crs:EPSG::5179">
    <gml:lowerCorner>953964.7841013006 1951189.550679219</gml:lowerCorner>
    <gml:upperCorn

In [7]:
tree = ElementTree.ElementTree(ElementTree.fromstring(response_text))

In [8]:
xmlns = dict()
for elem in response_text.split('>')[1].split('xmlns')[1:]:
    text = elem[1:].split(' ')[0]
    key, item = text.split('=')
    xmlns[key] = item.replace('"', '')
xmlns

{'xs': 'http://www.w3.org/2001/XMLSchema',
 'wfs': 'http://www.opengis.net/wfs',
 'gml': 'http://www.opengis.net/gml',
 'NSDI': 'http://10.1.17.66:6080/arcgis/services/opend/EiosSpceServiceWFS/MapServer/WFSServer',
 'xlink': 'http://www.w3.org/1999/xlink',
 'xsi': 'http://www.w3.org/2001/XMLSchema-instance'}

In [9]:
def remove_ns(tag):
    return tag.split('}')[-1]

In [10]:
df = pd.DataFrame(columns=['bldg_id', 'bldg_name', 'bldg_hg', 'ground_floor_co', 'undgrnd_floor_co', 'frst_regist_dt', 'shape'])
for bldg in tree.findall(f'.//gml:featureMember/NSDI:{TYPE_NAME}', xmlns):
#     print(remove_ns(bldg.tag))

#     bldg_id = bldg.attrib[f'{{{xmlns["gml"]}}}id']
    bldg_id = bldg.find('NSDI:GIS_IDNTFC_NO', xmlns).text
#     bldg_id = bldg.find('NSDI:BULD_IDNTFC_NO', xmlns).text
    print('bldg_id:', bldg_id)

    bldg_name = bldg.find('NSDI:BULD_NM', xmlns).text
    print('bldg_name:', bldg_name)

    bldg_hg = float(bldg.find('NSDI:BULD_HG', xmlns).text)
    print('bldg_hg:', bldg_hg)

    ground_floor_co = float(bldg.find('NSDI:GROUND_FLOOR_CO', xmlns).text)
    print('ground_floor_co:', ground_floor_co)

    undgrnd_floor_co = float(bldg.find('NSDI:UNDGRND_FLOOR_CO', xmlns).text)
    print('undgrnd_floor_co:', undgrnd_floor_co)

    frst_regist_dt = bldg.find('NSDI:FRST_REGIST_DT', xmlns).text
    print('frst_regist_dt:', frst_regist_dt)

    shape = bldg.find('NSDI:SHAPE//gml:posList', xmlns).text
    values = list(map(float, shape.split(' ')))
    print(values[::2], values[1::2])
#     print('shape:', shape)
    new_row =pd.DataFrame({
        'bldg_id' : bldg_id,
        'bldg_name' : bldg_name,
        'bldg_hg' : bldg_hg,
        'ground_floor_co' : ground_floor_co,
        'undgrnd_floor_co' : undgrnd_floor_co,
        'frst_regist_dt' : frst_regist_dt,
        'shape' : values
    })
    df = pd.concat([df,new_row])
    print('-'*100)
df.to_csv('LXtest.csv', index=False)

    

bldg_id: 1957197883484511206500000000
bldg_name: None
bldg_hg: 0.0
ground_floor_co: 4.0
undgrnd_floor_co: 1.0
frst_regist_dt: 2023-09-18T13:18:42
[953983.0348687812, 953970.9505950827, 953975.3033253743, 953987.7018315862, 953983.0348687812] [1951253.351064298, 1951257.475950824, 1951271.043490155, 1951267.165878487, 1951253.351064298]
----------------------------------------------------------------------------------------------------
bldg_id: 1969197892424511710900000000
bldg_name: None
bldg_hg: 0.0
ground_floor_co: 13.0
undgrnd_floor_co: 1.0
frst_regist_dt: 2023-09-18T13:18:42
[953978.349782769, 953993.4668927899, 954002.3422775562, 954000.9280257337, 953998.8622015098, 953977.5838479282, 953974.5886183943, 953974.7322243722, 953978.349782769] [1951321.359641164, 1951317.341681743, 1951314.982324149, 1951310.522934674, 1951304.035505502, 1951309.4287123, 1951310.194364611, 1951310.693478579, 1951321.359641164]
--------------------------------------------------------------------------

/tmp/ipykernel_3274861/2501707679.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,new_row])
